In [4]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
import nltk
from scipy import stats
import compound_splitter

In [15]:
def compute_values_move(refexps_dir, success_dir):
    splits = compound_splitter.load_dict('de_lower.dict')
    move_level_data = defaultdict(dict)
    move_level_align = []
    move_level_coherence = []
    success_types = set()
    playermove_level_align_speaker = defaultdict(list)
    playermove_level_coherence_speaker= defaultdict(list)

    for dialogue in listdir(refexps_dir):
        #extract dialogue type
        success_file = dialogue.replace('refexps', 'movesuccess')
        success_file = path.join(success_dir, success_file)
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue = path.join(refexps_dir, dialogue)
        #load referring expressions data
        dialogue = refexps.load_data(dialogue)
        success_moves = refexps.load_data(success_file)
        #move level
        dialogue_move = refexps.move_level(dialogue)
        #iterate over moves
        for move in dialogue_move:
            refs = move[1]
            #iterate over objects
            for obj in refs:
                #iterate over objects
                refobj = refs[obj]
                #SIMILARITY OF LEXICAL CONTENT INTO
                sim_align = []
                sim_align_speaker = defaultdict(list)
                sim_coherence_speaker = defaultdict(list)
                sim_coherence = []
                for ref in refobj:
                    speaker = ref[1]
                    sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj), language, splits)
                    if sim != 'undefined':
                        sim_align.append(sim)
                        sim_align_speaker[speaker].append(sim)
                sim_align = np.mean(sim_align)
                comb_ref = combinations(refobj,2)
                for comb in comb_ref:
                    if comb[0] != comb[1]:
                        sim = refexps.lex_sim(comb[0], comb[1], language, splits)
                        if sim != 'undefined':
                            sim_coherence.append(sim)
                            sim_coherence_speaker[comb[0][1]].append(sim)
                            sim_coherence_speaker[comb[1][1]].append(sim)
                sim_coherence = np.mean(sim_coherence)
                #movelevel coherence and alignment (for each object)
                if not np.isnan(sim_coherence):
                    move_level_coherence.append(sim_coherence)
                if not np.isnan(sim_align):
                    move_level_align.append(sim_align)
                for s in sim_align_speaker:
                    if not np.isnan(np.mean(sim_align_speaker[s])):
                        playermove_level_align_speaker[s].append(np.mean(sim_align_speaker[s]))
                for s in sim_coherence_speaker:
                    if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                        playermove_level_coherence_speaker[s].append(np.mean(sim_coherence_speaker[s]))
            values = [sim_coherence, sim_align, sim_align_speaker['e-utts'], sim_align_speaker['p-utts'], sim_coherence_speaker['e-utts'], sim_coherence_speaker['p-utts'] ]
            move_level_data[move[0]]['values'] = values
            success = success_moves[move[0]]
            move_level_data[move[0]]['success'] = success
    move_level_align_all = []
    move_level_coherence_all = []

    playermove_level_align_speaker_all = defaultdict(list)
    playermove_level_coherence_speaker_all = defaultdict(list)

    results_move = defaultdict(list)
    
    playermove_level_coherence = []
    for s in playermove_level_coherence_speaker:
        playermove_level_coherence += playermove_level_coherence_speaker[s]
        playermove_level_coherence_speaker_all[s] += playermove_level_coherence_speaker[s]
        results_move['Move coherence '+ s] = playermove_level_coherence_speaker[s]
    results_move['Move players coherence'] = playermove_level_coherence
    results_move['Move coherence'] = move_level_coherence

    playermove_level_align = []
    for s in playermove_level_align_speaker:
        playermove_level_align += playermove_level_align_speaker[s]
        playermove_level_align_speaker_all[s] += playermove_level_align_speaker[s]
        results_move['Move alignment '+ s] = playermove_level_align_speaker[s]
    results_move['Move players alignment'] = playermove_level_align
    results_move['Move alignment'] = move_level_align

    for value in results_move:
        dataset = results_move[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return results_move

In [19]:
de_dataset_dir = 'En_De_Dataset/De/RefExp'
de_success_dir = 'En_De_Dataset/De/Success'

en_dataset_dir = 'En_De_Dataset/En/RefExp'
en_success_dir = 'En_De_Dataset/En/Success'

all_dataset_dir = 'En_De_Dataset/All/RefExp'
all_success_dir = 'En_De_Dataset/All/Success'

results_de = compute_values_move(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_move(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_move(all_dataset_dir, all_success_dir)

	Move coherence: 
	Mean: 0.188896380461	Standard deviation: 0.287182912161
	Max value: 1.0	Min value: 0.0
	Move alignment e-utts: 
	Mean: 0.0783389198412	Standard deviation: 0.223947435992
	Max value: 1.0	Min value: 0.0
	Move alignment p-utts: 
	Mean: 0.0127828569427	Standard deviation: 0.0914058853529
	Max value: 1.0	Min value: 0.0
	Move players coherence: 
	Mean: 0.173983654506	Standard deviation: 0.282315728989
	Max value: 1.0	Min value: 0.0
	Move players alignment: 
	Mean: 0.0333440412273	Standard deviation: 0.149631685608
	Max value: 1.0	Min value: 0.0
	Move coherence p-utts: 
	Mean: 0.178262116651	Standard deviation: 0.279717995455
	Max value: 1.0	Min value: 0.0
	Move coherence e-utts: 
	Mean: 0.166282422645	Standard deviation: 0.286771576991
	Max value: 1.0	Min value: 0.0
	Move alignment: 
	Mean: 0.0191384745641	Standard deviation: 0.0830978312196
	Max value: 0.666666666667	Min value: 0.0



	Move coherence: 
	Mean: 0.182823534546	Standard deviation: 0.246655762255
	Max value: 1

In [21]:
for var in results_all:
    print var
    ttest =  stats.ttest_ind(results_de[var],results_en[var])
    if ttest[1] < 0.05:
        print str(ttest)+'\tSignificantly different'
    else:
        print ' Not significantly different'

Move coherence
 Not significantly different
Move alignment e-utts
 Not significantly different
Move alignment p-utts
 Not significantly different
Move players coherence
 Not significantly different
Move players alignment
 Not significantly different
Move coherence p-utts
 Not significantly different
Move coherence e-utts
 Not significantly different
Move alignment
 Not significantly different


In [11]:
def compute_values_game(refexps_dir, success_dir):
    splits = compound_splitter.load_dict('de_lower.dict')
    game_level_align = []
    game_level_coherence = []
    playergame_level_align_speaker = defaultdict(list)
    playergame_level_coherence_speaker = defaultdict(list)

    for dialogue in listdir(refexps_dir):
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue = refexps.load_data(path.join(refexps_dir, dialogue))
        dialogue_game = refexps.gamerun_level(dialogue)
        for obj in dialogue_game:
            refobj = dialogue_game[obj]
            sim_align = []
            sim_align_speaker = defaultdict(list)
            sim_coherence_speaker = defaultdict(list)
            sim_coherence = []
            for ref in refobj:
                speaker = ref[1]
                sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj), language, splits)
                if sim != 'undefined':
                    sim_align.append(sim)
                    sim_align_speaker[speaker].append(sim)
            sim_align = np.mean(sim_align)
            for s in sim_align_speaker[speaker]:
                sim_align_speaker[speaker] = np.mean(sim_align_speaker[speaker])
            comb_ref = combinations(refobj,2)
            for comb in comb_ref:
                if comb[0] != comb[1]:
                    sim = refexps.lex_sim(comb[0], comb[1], language, splits)
                    if sim != 'undefined':
                        sim_coherence.append(sim)
                        sim_coherence_speaker[comb[0][1]].append(sim)
                        sim_coherence_speaker[comb[1][1]].append(sim)
            sim_coherence = np.mean(sim_coherence)
            #gamelevel coherence and alignment (for each object)
            if not np.isnan(sim_coherence) :
                game_level_coherence.append(sim_coherence)
            if not np.isnan(sim_align):
                game_level_align.append(sim_align)
            for s in sim_align_speaker:
                if not np.isnan(np.mean(sim_align_speaker[s])):
                    playergame_level_align_speaker[s].append(np.mean(sim_align_speaker[s]))
            for s in sim_coherence_speaker:
                if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                    playergame_level_coherence_speaker[s].append(np.mean(sim_coherence_speaker[s]))

    game_level_align_all = []
    game_level_coherence_all = []

    playergame_level_align_speaker_all = defaultdict(list)
    playergame_level_coherence_speaker_all = defaultdict(list)

    results_game = defaultdict(list)

    playergame_level_coherence = []
    for s in playergame_level_coherence_speaker:
        playergame_level_coherence += playergame_level_coherence_speaker[s]
        playergame_level_coherence_speaker_all[s] += playergame_level_coherence_speaker[s]
        results_game['game coherence '+ s] = playergame_level_coherence_speaker[s]
    results_game['game players coherence'] = playergame_level_coherence
    results_game['game coherence'] = game_level_coherence
    playergame_level_align = []
    for s in playergame_level_align_speaker:
        playergame_level_align += playergame_level_align_speaker[s]
        playergame_level_align_speaker_all[s] += playergame_level_align_speaker[s]
        results_game['game alignment '+ s] = playergame_level_align_speaker[s]
    results_game['game players alignment'] = playergame_level_align
    results_game['game alignment'] = game_level_align
    for value in results_game:
        dataset = results_game[value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tStandard deviation: '+ str(np.std(dataset))
        print '\tMax value: ' + str(max(dataset)) + '\tMin value: '+ str(min(dataset))
    return results_game

In [23]:

results_de = compute_values_game(de_dataset_dir, de_success_dir)
print '\n\n'
results_en = compute_values_game(en_dataset_dir, en_success_dir)
print '\n\n'
results_all = compute_values_game(all_dataset_dir, all_success_dir)

	game players coherence: 
	Mean: 0.111700272806	Standard deviation: 0.173066823379
	Max value: 1.0	Min value: 0.0
	game coherence e-utts: 
	Mean: 0.113244535277	Standard deviation: 0.172929050429
	Max value: 1.0	Min value: 0.0
	game alignment: 
	Mean: 0.0193176036799	Standard deviation: 0.0696030133704
	Max value: 0.533333333333	Min value: 0.0
	game alignment p-utts: 
	Mean: 0.0106359878023	Standard deviation: 0.0496096345117
	Max value: 0.5	Min value: 0.0
	game coherence p-utts: 
	Mean: 0.110818877717	Standard deviation: 0.173139234795
	Max value: 1.0	Min value: 0.0
	game players alignment: 
	Mean: 0.0337033816161	Standard deviation: 0.128962167906
	Max value: 1.0	Min value: 0.0
	game coherence: 
	Mean: 0.114203590182	Standard deviation: 0.175171250337
	Max value: 1.0	Min value: 0.0
	game alignment e-utts: 
	Mean: 0.08114036083	Standard deviation: 0.205977917528
	Max value: 1.0	Min value: 0.0



	game players coherence: 
	Mean: 0.143861468595	Standard deviation: 0.196350067651
	Max va

In [24]:
for var in results_all:
    print var
    ttest =  stats.ttest_ind(results_de[var],results_en[var])
    if ttest[1] < 0.05:
        print str(ttest)+'\tSignificantly different'
    else:
        print ' Not significantly different'

game players coherence
Ttest_indResult(statistic=-2.0444991328327111, pvalue=0.041371339619496031)	Significantly different
game coherence e-utts
 Not significantly different
game alignment
 Not significantly different
game alignment p-utts
 Not significantly different
game coherence p-utts
Ttest_indResult(statistic=-2.3652161885405674, pvalue=0.018572248905435813)	Significantly different
game players alignment
 Not significantly different
game coherence
Ttest_indResult(statistic=-2.4247606529011838, pvalue=0.015816939112154101)	Significantly different
game alignment e-utts
 Not significantly different


game players coherence and game coherence is always significantly higher at the move level

In [1]:
import pickle
refexps = pickle.load(open('Dataset/Disruption_with_Noise_Corpus/Success/20070201_run1pento_nonoise_success.p', 'rb'))

IOError: [Errno 2] No such file or directory: 'Dataset/Disruption_with_Noise_Corpus/Success/20070201_run1pento_nonoise_success.p'

In [6]:
refexps

[((4, 251),
  {'e-utts': {'1l': {'location': [('a short bit', (61, 63), 'indef_np'),
      ('the short bit', (70, 72), 'def_np'),
      ('a block of four', (224, 227), 'indef_np')],
     'part_of_piece': [('a short bit', (61, 63), 'indef_np'),
      ('the short bit', (70, 72), 'def_np'),
      ('a block of four', (224, 227), 'indef_np')],
     'piece': [('a short bit', (61, 63), 'indef_np'),
      ('the short bit', (70, 72), 'def_np'),
      ('a block of four', (224, 227), 'indef_np')]}},
   'p-utts': {'1l': {'desc_piece': [('the short l bottom bit',
       (29, 33),
       'def_np'),
      ('two squares', (36, 37), 'quant_np'),
      ('the short bit', (44, 46), 'def_np')],
     'location': [('the short l bottom bit', (29, 33), 'def_np'),
      ('two squares', (36, 37), 'quant_np'),
      ('the short bit', (44, 46), 'def_np')],
     'part_of_piece': [('the short l bottom bit', (29, 33), 'def_np'),
      ('two squares', (36, 37), 'quant_np'),
      ('the short bit', (44, 46), 'def_np')]